In [1]:
from get_financial import *

In [2]:
for gicode in ['093640'] :
    post_info = get_html(gicode)
    # create_post(post_info)

20230625_093640_케이알엠.html file has been created.
[A.I] 케이알엠:주식 재무 분석 & 적정주가 계산_2023/06/25
